In [ ]:
from time import time
import gc

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Input, Dropout
from tensorflow.keras.optimizers import Adam, SGD

In [ ]:
batch_size = 8192
epochs = 100

In [ ]:
input_size = 22 + 16 * 28
output_size = 1

x_train = np.load('/home/vova/Downloads/hlt_client/hlt_client/data/train_comb_features_half.npy')
y_train = np.load('/home/vova/Downloads/hlt_client/hlt_client/data/train_comb_outputs_half.npy')
x_test = np.load('/home/vova/Downloads/hlt_client/hlt_client/data/test_comb_features_half.npy')
y_test = np.load('/home/vova/Downloads/hlt_client/hlt_client/data/test_comb_outputs_half.npy')


In [ ]:
model = Sequential([
    Input(shape=(input_size)),
    Dense(500, kernel_initializer='normal'),
    Activation('relu'),
    Dense(500, kernel_initializer='normal'),
    Activation('relu'),
    Dense(500, kernel_initializer='normal'),
    Activation('relu'),
    Dense(500, kernel_initializer='normal'),
    Activation('relu'),
    Dense(500, kernel_initializer='normal'),
    Activation('relu'),
    Dense(500, kernel_initializer='normal'),
    Activation('relu'),
    Dense(1, kernel_initializer='normal'),
    Activation('linear'),
])

model.summary()

In [ ]:
model.compile(
    optimizer=Adam(lr=0.01, clipnorm=0.01),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=['mae', 'mse'],
)
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir=f"../runs/{time()}", update_freq='epoch')
]
gc.collect()

In [ ]:

model.fit(
    x=x_train,
    y=y_train[:,1],
    validation_data=(x_test, y_test[:,1]),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=callbacks
)

In [ ]:
i = 20004
predicted = list(model.predict(x_train[i].reshape(1,-1))[0])
expected = list(y_train[i])
print(f'{expected=}\n{predicted=}')
predicted2 = list(model.predict(x_test[i].reshape(1,-1))[0])
expected2 = list(y_test[i])
print(f'{expected2=}\n{predicted2=}')

In [ ]:
model.save('weights.h5')

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
gc.collect()

In [ ]:
reg = RandomForestRegressor(max_depth=10, verbose=True, n_jobs=8)
reg.fit(x_train[1:20000], y_train[1:20000])

In [ ]:
print(f'train: {reg.score(x_train, y_train)}')
print(f'test: {reg.score(x_test, y_test)}')

In [ ]:
i = 20004
predicted = list(reg.predict(x_train[i].reshape(1,-1))[0])
expected = list(y_train[i])
print(f'{expected=}\n{predicted=}')
predicted2 = list(reg.predict(x_test[i].reshape(1,-1))[0])
expected2 = list(y_test[i])
print(f'{expected2=}\n{predicted2=}')

In [ ]:
import matplotlib.pyplot as plt
from feature_extractor import Feature, Move, FeaturePlanets
start = 110_000
x_data = x_train[start:5000+start,22 + FeaturePlanets.angle]
# y_data0 = y_train[start:5000+start,Move.magnitude]
y_data1 = y_train[start:5000+start,Move.angle]
mask = y_data1[x_data < 400]
# [x_data < 400]

# plt.plot(x_data[x_data < 400][mask < 361], y_data0[x_data < 400][mask < 361], 'bo')
# plt.show()
plt.plot(x_data[x_data < 400][mask < 361], y_data1[x_data < 400][mask < 361], 'bo')
plt.show()

print(x_data[:100])